**Extracción de Tablas de Premios Oscar (link) con BeautifulSoup:**
- Fecha de la ceremonia.
- Mejor película.
- Mejor director.
- Mejor actor.
- Mejor actriz.

**NOTA:** La información deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. 

In [1]:
# Importamos librerías.
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
# Definimos la url.
url_oscars = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"

# Hacemos la llamada a la url.
res_oscars = requests.get(url_oscars)

# Comprobamos el código de respuesta.
res_oscars.status_code
print(f"La respuesta de la llamada es: {res_oscars.status_code}")

In [ ]:
# Mostramos el contenido.
contenido = res_oscars.content
contenido

In [ ]:
# Usamos BeautifulSoup para construir el objeto.
sopa_oscars = BeautifulSoup(contenido, "html.parser")
print(sopa_oscars.prettify())

In [ ]:
# Buscamos las tablas de donde extraeremos la info relevante.
tablas = sopa_oscars.find_all("table")
print("El número de tablas que tenemos en la página web es:", len(tablas))

In [ ]:
# Comprobamos el contenido de las tablas para asegurarnos de cuál escogemos según los datos que necesitamos.
print("Tabla : \n ", tablas[2])
print("\n---------------------\n")

In [ ]:
# Hacemos una variable para almacenar nuestra tabla.
tabla_wikipedia = tablas[2]
tabla_wikipedia

In [ ]:
# Extraemos los encabezados de los datos relevantes para facilitar la posterior estructura de DataFrame.
lista_encabezados= tabla_wikipedia.find_all("th")[1:6]
lista_encabezados

In [ ]:
# Iteramos por la lista obtenida en el paso anterior para extraer el texto de cada encabezado.
encabezados_wikipedia =  [columna.text.replace("\n", "") for columna in lista_encabezados]
encabezados_wikipedia

In [ ]:
# Extraemos los filas de nuestra variable usando el método .find_all() excepto la fila de encabezados
filas_wikipedia = tabla_wikipedia.find_all("tr")[1:]
filas_wikipedia 

In [ ]:
print("La cantidad de filas que hemos extraído de la tabla es:", len(filas_wikipedia))

print("\n----------------------\n")
print("El contenido que tenemos en la primera fila es:\n", filas_wikipedia[0])

In [ ]:
import re
# Patrón de regex que detecta los meses seguidos de "de" sin espacio
patron = r"(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)de"

# Creamos una lista para almacenar los resultados obtenidos de la lista filas wikipedia.
resultados_wikipedia = []

# Iteramos por la lista
for fila in filas_wikipedia:
    # añadimos cada elemento a la lista que hemos creado previamente. 
    fila_texto = fila.text
    elementos_fila = fila_texto.split("\n")[3:12:2]

    #  Usamos regex para corregir el formato de la fecha en la primera posición
    elementos_fila[0] = re.sub(patron, r"\1 de", elementos_fila[0])
    
    resultados_wikipedia.append(elementos_fila)
 
# Imprimimos los resultados obtenidos después de iterar por la lista.
print("Los resultados de iterar por la lista son:\n", resultados_wikipedia)

In [ ]:
# DEFINIMOS LA FUNCIÓN
def sacar_datos_oscar(url):

    # Hacemos la llamada a la url.
    res_oscars = requests.get(url_oscars)

    # Comprobamos el código de respuesta.
    print(f"La respuesta de la llamada es: {res_oscars.status_code}")

    # Usamos BeautifulSoup para construir el objeto
    contenido = res_oscars.content
    sopa_oscars = BeautifulSoup(contenido, "html.parser")

    # Buscamos las tablas de donde extraeremos la info relevante.
    tablas = sopa_oscars.find_all("table")
    
    # Hacemos una variable para almacenar nuestra tabla.
    tabla_wikipedia = tablas[2]

    encabezados_wikipedia =  [columna.text.replace("\n", "") for columna in lista_encabezados]

    # Extraemos los filas excepto la fila de encabezados
    filas_wikipedia = tabla_wikipedia.find_all("tr")[1:]

    # Creamos una lista para almacenar los resultados obtenidos de la lista filas wikipedia
    resultados_wikipedia = []

    # Iteramos por la lista
    for fila in filas_wikipedia:
        # añadimos cada elemento a la lista que hemos creado previamente. 
        elementos_fila = fila.text.split("\n")[3:12:2]
        #  Usamos regex para corregir el formato de la fecha en la primera posición
        elementos_fila[0] = re.sub(patron, r"\1 de", elementos_fila[0])
        
        resultados_wikipedia.append(elementos_fila)
    
    return encabezados_wikipedia, resultados_wikipedia

In [ ]:
encabezados_wikipedia_final, resultados_wikipedia_final = sacar_datos_oscar(url_oscars)

In [ ]:
encabezados_wikipedia_final

In [ ]:
df_oscars = pd.DataFrame(resultados_wikipedia_final)
df_oscars

In [ ]:
df_oscars.columns = encabezados_wikipedia_final

In [ ]:
df_oscars.head()

In [ ]:
# Guardamos la información y no tener que runear todo el código.
df_oscars.to_csv("datos_oscars.csv")